In [1]:
%load_ext autoreload
%autoreload 2

import sys, os

# Go up one directory from `b/` to project root
root_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(root_dir)

In [2]:
import logging
from neurosurfer.models.chat_models.transformers import TransformersModel
from neurosurfer.models.chat_models.unsloth import UnslothModel
from neurosurfer import config 
import torch
torch.cuda.empty_cache()

# falcon-7b-instruct
# "model_name": "/home/nomi/workspace/Model_Weights/Qwen3-8B-unsloth-bnb-4bit",
# "model_name": "/home/nomi/workspace/Model_Weights/falcon-7b-instruct",


DEFAULT_TRANSFORMERS_MODEL_PARAMS = dict({
    "model_name": "/home/nomi/workspace/Model_Weights/Qwen3-8B-unsloth-bnb-4bit",
    "max_seq_length": 4096,
    "load_in_4bit": True,
    "enable_thinking": False,
    "reasoning_effort": "low",
    "add_special_tokens": False,
    "verbose": False
})

LOGGER = logging.getLogger()
LLM = TransformersModel(
    **DEFAULT_TRANSFORMERS_MODEL_PARAMS,
    stop_words=["Observation:"],
    logger = logging.getLogger(),
)

╔══════════════════════════════════════════════════════════════════╗
║                                                                  ║
║ ▓▓▓▓▓   ▓▓▓▓                                  ▓▓▓                ║
║  ▓▓ ▓▓   ▓▓  ▓▓▓▓ ▓  ▓ ▓ ▓ ▓▓▓▓ ▓▓▓ ▓  ▓ ▓ ▓  ▓   ▓▓▓▓ ▓ ▓       ║
║  ▓▓  ▓▓  ▓▓  ▓▁▁▓ ▓  ▓ ▓▓▏ ▓  ▓ ▓▁  ▓  ▓ ▓▓▏ ▓▓▓  ▓▁▁▓ ▓▓        ║
║  ▓▓   ▓▓ ▓▓  ▓    ▓  ▓ ▓   ▓  ▓   ▓ ▓  ▓ ▓    ▓   ▓    ▓         ║
║ ▓▓▓▓   ▓▓▓▓▓ ▓▓▓▓ ▓▓▓▓ ▓   ▓▓▓▓ ▓▓▓ ▓▓▓▓ ▓    ▓   ▓▓▓▓ ▓         ║
║ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ║
║ Orchestrate Agents - RAG - SQL Tools - Multi-LLM - FastAPI Ready ║
║ Faster builds, clearer flows, production-first                   ║
║                                                                  ║
║ Version: 0.1.3 | Python: 3.11.13                                 ║
║ OS: Linux 6.14.0-36-generic (x86_64)                             ║
║ Torch: 2.7.1+cu126   CUDA: yes (12.6)                            ║
║ MPS: no (built: False)          

/home/nomi/anaconda3/envs/LLMs/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO     | 2025-12-11 12:16:07 | config.py:<module> | PyTorch version 2.7.1+cu126 available.


/home/nomi/anaconda3/envs/LLMs/lib/python3.11/importlib/__init__.py:126: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  return _bootstrap._gcd_import(name[level:], package, level)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 12-11 12:16:08 [__init__.py:241] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO     | 2025-12-11 12:16:11 | transformers.py:init_model | Initializing Transformers model.
WARNING  | 2025-12-11 12:16:11 | transformers.py:init_model | Model is already quantized. Ignoring load_in_4bit=True.
INFO     | 2025-12-11 12:16:11 | modeling.py:get_balanced_memory | We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.65s/it]

INFO     | 2025-12-11 12:16:16 | transformers.py:init_model | Transformers model initialized successfully.


In [3]:
# streaming response example
from IPython.display import display, Markdown, clear_output

system_prompt = """You are a joke teller"""
user_prompt = """Tell me a short and light-hearted joke."""

stream_response = LLM.ask(
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    stream=True
)

# print(stream_response.choices[0].message.content)
md_display = display(Markdown(""), display_id=True)
for chunk in stream_response:
    chunk = chunk.choices[0].delta.content or ""
    print(chunk, flush=True, end="")

Why don't skeletons fight each other?  
Because they don’t have the *guts*! 😄

## Agent Test

**Normal Response**

In [4]:
# agent normal response
from neurosurfer.agents import Agent, AgentConfig
# from neurosurfer.tracing import RichTracer
from pydantic import BaseModel

agent_config = AgentConfig(
    strict_tool_call=True,
    return_stream_by_default=True,
    temperature=0.7,
    max_new_tokens=4096,
)
agent = Agent(llm=LLM, config=agent_config, log_traces=True)

# normal response
print("Normal Response:")
agent_response = agent.run(user_prompt="What is AI (one line)?", stream=False)
# print(agent_response.response)

# # streaming response
# print("\n\nStreaming Response:")
# for c in agent.run(user_prompt="What are top 3 applications of AI (one line)?").response:
#     print(c, flush=True, end="")


Normal Response:
🧠 Thinking...

[main_agent] Tracing Start!
 ▶ [1][step.agent] agent_id='main_agent' label='agent.run'
     ▶ [2][step.llm.call] agent_id='main_agent' label='agent.free_text_call'
     ◀ [2][step.llm.call] agent_id='main_agent' label='agent.free_text_call' took 0.818s; error=False
 ◀ [1][step.agent] agent_id='main_agent' label='agent.run' took 0.819s; error=False
[main_agent] Tracing End!

Final response:
AI, or Artificial Intelligence, is the simulation of human intelligence in machines that are programmed to think, learn, and make decisions.


**Structured Response**

In [5]:
# Structured Response examples
class AIApplication(BaseModel):
    title: str
    description: str

class AI(BaseModel):
    definition: str
    history: str
    modern_frameworks: str
    applications: list[AIApplication]

user_query = "What is AI and list 3 of its top application, and 3 concerns."
agent_response = agent.run(user_prompt=user_query, output_schema=AI)

print("\n\nStructured Response:")
print(agent_response.response.json_obj)  

WARNING  | 2025-12-11 12:16:19 | agent.py:run     | `output_schema` provided with `stream=True`; forcing non-streaming structured output.
🧠 Thinking...

[main_agent] Tracing Start!
 ▶ [1][step.agent] agent_id='main_agent' label='agent.run'
     ▶ [2][step.llm.call] agent_id='main_agent' label='agent.structured_call.first_pass'
     ◀ [2][step.llm.call] agent_id='main_agent' label='agent.structured_call.first_pass' took 5.007s; error=False
 ◀ [1][step.agent] agent_id='main_agent' label='agent.run' took 5.009s; error=False
[main_agent] Tracing End!



Structured Response:
{
  "definition": "Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think, learn, and make decisions.",
  "history": "The concept of AI dates back to the 1950s, with the term 'artificial intelligence' coined in 1956. Early developments included rule-based systems and expert systems, while recent advancements have focused on machine learning and neural network

In [6]:
# agent_response.traces.steps[1].model_dump()

In [7]:
# from typing import List
# from neurosurfer.server.db.db import SessionLocal
# from neurosurfer.server.db.models import NSFile
# import re

# ALLOWED_NAME = re.compile(r"[^a-zA-Z0-9._-]+")

# def _collection_for(user_id: int, thread_id: int) -> str:
#     return _safe_collection_name(f"ns_vdb_u{user_id}_t{thread_id}")

# def _safe_collection_name(s: str) -> str:
#     s = ALLOWED_NAME.sub("_", s).strip("._-")
#     return s or "ns_vdb_default"


# user_id = 1
# thread_id = 1
# message_id = 1
# collection_name = _collection_for(user_id, thread_id)

# db = SessionLocal()
# files = (
#     db.query(NSFile)
#     .filter(
#         NSFile.user_id == user_id,
#         NSFile.thread_id == thread_id,
#         NSFile.collection == collection_name,
#         NSFile.ingested == True
#     )
#     .all()
# )
# files_block_lines: List[str] = []
# for f in files:
#     flag = "yes" if message_id and f.message_id == message_id else "no"
#     files_block_lines.append(
#         f"Filename: {f.filename}\n"
#         f"Attached_to_current_message: {flag}\n"
#         f"Summary:\n{f.summary or '(no summary)'}\n"
#     )
# files_block = "\n\n".join(files_block_lines)

# print(files_block)




## SERVER SIDE

In [18]:
from neurosurfer.server.app import NeurosurferApp
from neurosurfer.agents.react import ReActConfig
from neurosurfer.agents.code import CodeAgentConfig
from neurosurfer.server.services.chat_orchestration import MainWorkflowConfig
from neurosurfer.config import config

# Create app instance
ns = NeurosurferApp(
    app_name="Neurosurfer API",
    api_keys=[],
    enable_docs=True,
    cors_origins=["*"],
    host="0.0.0.0",
    port=8000,
    reload=config.app.reload,
    log_level=config.app.logs_level,
    workers=config.app.workers
)

# Initialize models
ns.register_model(
    model=LLM,
    family="llama",
    provider="Unsloth",
    description="Proxy to Llama"
)

# Initialize RAG
ns._init_rag(embedder=ns._embedder, llm=ns._llm)

# Initialize agent
temperature = 1.0
max_new_tokens = 8000
ns._init_chat_workflow(
    config=MainWorkflowConfig(  
        default_language="English",
        default_answer_length="detailed",
        enable_rag=True,
        enable_code=True,
        log_traces=True,
        max_context_chars=16000,
        max_history_chars=12000,
        temperature=temperature
    ),
    code_agent_config=CodeAgentConfig(
        temperature=temperature,
        max_new_tokens=max_new_tokens,
        mode="analysis_only"
    )
)

INFO     | 2025-12-11 12:27:10 | SentenceTransformer.py:__init__ | Use pytorch device_name: cuda:0
INFO     | 2025-12-11 12:27:10 | sentence_transformer.py:__init__ | SentenceTransformer embedding model initialized.
INFO     | 2025-12-11 12:27:10 | app.py:__init__  | Default embedder: intfloat/e5-large-v2 successfully initialized. Replace it by calling app.register_model(model: BaseEmbedder)
INFO     | 2025-12-11 12:27:10 | db.py:init_db    | Database initialized successfully in sqlite:////home/nomi/.cache/Neurosurfer/app-storage/app.db
INFO     | 2025-12-11 12:27:10 | app.py:register_model | LLM /home/nomi/workspace/Model_Weights/Qwen3-8B-unsloth-bnb-4bit successfully registered.
WARNING  | 2025-12-11 12:27:10 | agent.py:__init__ | No vectorstore provided to RAGAgent, using default ChromaVectorStore. Initializing default collection `neurosurfer-rag-agent`
[Init] ChromaVectorStore initialized with collection: neurosurfer-rag-agent
INFO     | 2025-12-11 12:27:10 | orchestrator.py:__init

In [32]:
# test the server
# Prepare inputs

user_query = "Give me 5 students names starting with letter A."
# user_query = "Hi how are you? Explain what is AI in 2 lines."
# user_query = "Hallo, wie geht es dir? Erkläre in zwei Sätzen, was KI ist."
# Minimal chat history excluding system messages, max 10
# num_recent = 10
# conversation_messages = args.messages.converstaion
# chat_history = conversation_messages[-num_recent:-1]

# Model call (stream or non-stream handled by router)
# kwargs = {"temperature": args.temperature, "max_new_tokens": args.max_tokens}

user_id = 1
thread_id = 1
message_id = 1
has_files_message = False

agent_return = ns.run_agent(
    user_id=user_id,
    thread_id=thread_id,
    message_id=message_id,
    user_query=user_query,
    has_files_message=has_files_message,
)

for chunk in agent_return.final_answer_stream:
    ...
    # chunk = chunk.choices[0].delta.content or ""
    # print(chunk, end="", flush=True)

🧠 Thinking...

[main_chat_workflow] Tracing Start!
 ▶ [1][step.main_chat_workflow] agent_id='main_chat_workflow' label='main_chat_workflow.run'
     ▶ [2][step.llm.call] agent_id='gate_llm' label='gate.decide'
     ◀ [2][step.llm.call] agent_id='gate_llm' label='gate.decide' took 3.300s; error=False
    Route: code
    Optimized Query: Extract the names of 5 students starting with the letter 'A' from the Student Degree College Data Small.csv file.
    Query Language Detected: english
    Use Files: ['Student Degree College Data Small.csv']
    Clarification Question: None
    Reason: The request involves filtering and extracting specific data from a structured file, which requires the code agent to process the data.
    INFO: Running code pipeline
    🧠 Thinking...

    [CodeAgent] Tracing Start!
     ▶ [3][step.react_agent] agent_id='CodeAgent' label='react_agent.run'
         ▶ [4][step.llm.call] agent_id='CodeAgent' label='react_agent.loop.reason_llm_call'
            Thought: I nee

In [19]:

# agent_return.traces.model_dump()
agent_return.traces.steps

# print(agent_return.code_context.context_block)


[TraceStep(step_id=1, kind='main_chat_workflow', label='main_chat_workflow.run', node_id=None, agent_id='main_chat_workflow', started_at=1765440989.3446496, duration_ms=17168, inputs={'agent_type': 'MainChatWorkflow', 'response_type': 'streaming', 'has_files_message': False}, outputs={}, meta={}, ok=True, error=None, logs=[TraceLog(ts=1765440992.6312563, message="Route: code\nOptimized Query: Extract the names of students starting with the letter 'A' from the Student Degree College Data Small.csv file and return the top 5 names.\nQuery Language Detected: english\nUse Files: ['Student Degree College Data Small.csv']\nClarification Question: None\nReason: The request requires processing structured data to extract specific information, which is best handled by a code agent.\n", data={}, type='thought'), TraceLog(ts=1765440992.633913, message='Running code pipeline', data={}, type='info'), TraceLog(ts=1765441006.511969, message='Code pipeline completed with retrieved context of size 3171 c

In [31]:
ns.code_agent.tool_calls[4].__dict__

{'tool': 'python_execute',
 'inputs': {'task': "Select the top 5 names from the filtered list of students starting with 'A', using the correct column name 'name'."},
 'final_answer': False,
 'memory_keys': ['python_last_error'],
 'rationale': None,
 'output': '<special_token>\n<__code__>```python\nimport pandas as pd\n\n# Load the CSV file\nfile_path = files["Student Degree College Data Small.csv"]["path"]\nstudents = pd.read_csv(file_path)\n\n# Filter students whose names start with \'A\'\nfiltered_students = students[students[\'name\'].str.startswith(\'A\')]\n\n# Select the top 5 names\ntop_5_names = filtered_students[\'name\'].head(5).tolist()\n\nresult = top_5_names\n```</__code__>\n<special_token>\nHere are the results from executing Python code for your task:\n\n[\n  "Ankur Das",\n  "Abhijit Das",\n  "Abhik Bhattacharjee",\n  "Anirban Pal",\n  "Ankita Mandal"\n]'}